In [10]:
import re
import string

import pandas as pd

In [11]:
name = 'Book1.xlsx - Raw Text.csv'
df = pd.read_csv('docs/{}'.format(name), header=None)

In [12]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [13]:
# Use nice Pandas functionality to create a new column, without spaces
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

def replace_punctuation_and_spaces(text):
    text = remove_punctuation(text)
    # Space not a part of string.punctuation, so we do it manually.
    text = text.replace(' ', '')
    return text

# Count words and characters
def count_words(text):
    text = remove_punctuation(text)
    text = remove_numbers(text)
    # Split on whitespace.
    split_text = text.split()
    return len(split_text)

def remove_numbers(text):
    for number in '1234567890':
        text = text.replace(number, '')
    return text

def format_speech(spreadsheet_row):
    text = spreadsheet_row.iloc[0]
    index = spreadsheet_row['index']
    # \n is the newline sign, split the text on that.
    splitted_text = text.split('\n')
    
    # Remove empty lines
    no_empty_lines = [t for t in splitted_text if t.strip() != '']

    # Remove whitespace in the beginning of a line
    no_initial_whitespace = [t.lstrip() for t in no_empty_lines]
    
    # Put into a Pandas dataframe
    df = pd.DataFrame({'text': no_initial_whitespace})

    # Add empty columns
    df['empty_1'] = ''
    df['empty_2'] = ''
    df['empty_3'] = ''

    
    df['no_formatting'] = df.apply(lambda row: replace_punctuation_and_spaces(row['text']), axis=1)
    # count words
    df['word_count'] = df.apply(lambda row: count_words(row['text']), axis=1)    
    
    # Count characters
    df['char_count'] = df.apply(lambda row: len(remove_numbers(row['no_formatting'])), axis=1)
    
    df.to_csv('formatted/formatted_speech_{}.csv'.format(index))

In [14]:
# Does "format_speech" once for every row.
df['index'] = df.index
df.apply(format_speech, axis=1);

In [16]:
# Sanity check
# pd.read_csv('formatted/formatted_speech_0.csv')